In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import pydae.svg_tools as svgt
%config InlineBackend.figure_format = 'svg'

In [2]:
%matplotlib widget

In [3]:
from pydae import ssa
from lead_lag import lead_lag_class

In [4]:
syst = lead_lag_class()

In [5]:

syst.initialize([{    # parameters setting
                   # initial desired angle = 0º
                }],-1)                  # here -1 means that -1 is considered as initial gess for
                                        # dynamic and algebraic states

True

In [6]:
syst.report_y()

z_l   =  0.00


In [7]:
syst.simulate([{'t_end':1.0, 'u_l':0.0, 'T_1':0.1, 'T_2':0.1},
               {'t_end':5.0, 'u_l':0.5}],0);

In [8]:
fig, axes = plt.subplots(nrows=1,ncols=1, figsize=(6, 3), dpi=100)

axes.plot(syst.T, syst.get_values('z_l'), label=f'z_l')
axes.set_ylabel('$\\theta (º)$')
axes.set_xlabel('Time (s)')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
Δt = 0.001
syst = lead_lag_class()
times = np.arange(0,5,Δt)
syst.initialize([{'u_l':0.0, 'T_1':1.0, 'T_2':0.1}],0)

for t in times:
    syst.run([{'t_end':t,'u_l':np.sin(1*np.pi*2*t)}])
    
syst.post();


In [10]:
fig, axes = plt.subplots(nrows=1,ncols=1, figsize=(6, 3), dpi=100)

axes.plot(syst.T, syst.get_values('u_l'), label=f'u_l')
axes.plot(syst.T, syst.get_values('z_l'), label=f'z_l')
axes.legend()
axes.set_xlabel('Time (s)')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
import ipywidgets
%matplotlib widget

In [19]:
plt.ioff()
plt.clf()
syst = lead_lag_class()
syst.initialize([{'u_l':0.0, 'T_1':1.0, 'T_2':0.1}],0)
fig, axes = plt.subplots(nrows=1,ncols=1, figsize=(6, 3), dpi=100)

line_u_l = axes.plot(syst.T, syst.get_values('u_l'), label=f'u_l')
line_z_l = axes.plot(syst.T, syst.get_values('z_l'), label=f'z_l')
axes.legend()
axes.set_xlabel('Time (s)')
fig.tight_layout()



axes.set_ylim((-2,2))
axes.set_xlim((0,50))

axes.grid(True)
axes.legend(loc='best')
fig.tight_layout()


sld_T_1  = ipywidgets.FloatSlider(orientation='horizontal',description = "$\sf T_1$", 
                                value=0.1, min=-0.5,max= 2.0, 
                                step=0.01,continuous_update=False)


sld_T_2  = ipywidgets.FloatSlider(orientation='horizontal',description = "$\sf T_2$", 
                                value=0.1, min=0.05,max= 10.0, 
                                step=0.01,continuous_update=False)

sld_freq  = ipywidgets.FloatSlider(orientation='horizontal',description = "$\sf freq$", 
                                value=1.0, min=0.1,max= 2.0, 
                                step=0.1)

ckbox_gain = ipywidgets.Checkbox(
    value=False,
    description='Gain compensation',
    disabled=False
)

txt_phase = ipywidgets.Text(
    value='1∠0º',
    placeholder='Type something',
    description='gain∠phase:',
    disabled=False)

drop_signal = ipywidgets.Dropdown(
    options=['sin(wt)', 'step'],
    value='sin(wt)',
    description='Input:',
    disabled=False,
)



Δt = 0.01
times = np.arange(0,5,Δt)

def update(change):
    
    T_1 = sld_T_1.value
    T_2 = sld_T_2.value
    freq  = sld_freq.value
    
    cplx_ = (1j*2*np.pi*freq*T_1 + 1)/(1j*2*np.pi*freq*T_2 + 1)
    phase_wo = np.angle(cplx_,deg=True)
    
    gain_wo = 1.0
    if ckbox_gain.value:
        gain_wo = np.abs(cplx_)
        
    
    txt_phase.value = f'{np.abs(cplx_):4.1f}∠{phase_wo:4.1f}º'
    
    syst.initialize([{'u_l':0.0, 'T_1':T_1, 'T_2':T_2}],0)
    
    if drop_signal.value == 'sin(wt)':
        for t in times:
            syst.run([{'t_end':t,'u_l':np.sin(2*np.pi*freq*t)}])
    if drop_signal.value == 'step':
        syst.run([{'t_end':50,'u_l':1.0, 'Dt':0.01}])
        

    syst.post();

    line_u_l[0].set_data(syst.T, syst.get_values('u_l'))
    line_z_l[0].set_data(syst.T, syst.get_values('z_l')/gain_wo)

    fig.canvas.draw_idle()
      
sld_T_1.observe(update, names='value')
sld_T_2.observe(update, names='value')
sld_freq.observe(update, names='value')
ckbox_gain.observe(update, names='value')
drop_signal.observe(update, names='value')

layout_row1 = ipywidgets.HBox([fig.canvas])
layout_row2 = ipywidgets.HBox([sld_T_1,sld_T_2, txt_phase])
layout_row3 = ipywidgets.HBox([sld_freq,ckbox_gain,drop_signal])
layout = ipywidgets.VBox([layout_row1,layout_row2,layout_row3])
layout